In [1]:

import requests
import pandas as pd
import bs4 
import numpy as np

Get wiki ticker

In [2]:

def get_wiki_ticker():
    wiki_page = requests.get("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies").text
    soup = bs4.BeautifulSoup(wiki_page, 'html.parser')
    table = soup.find("table", {"class":"wikitable sortable"})

    ticker = []
    Sector = []
    SubSector = []
    HqLocation = []
    Founded = []

    # First row is header
    for row in table.findAll('tr')[1:]:
        ticker.append(row.findAll('td')[0].text)
        Sector.append(row.findAll('td')[3].text)
        SubSector.append(row.findAll('td')[4].text)
        HqLocation.append(row.findAll('td')[5].text)
        Founded.append(row.findAll('td')[8].text)

    ticker = list(map(lambda s: s.strip(), ticker))
    Founded = list(map(lambda s: s.strip(), Founded))

    ticker = pd.DataFrame(ticker,columns=['ticker'])
    sector = pd.DataFrame(Sector,columns=['sector'])
    subSector = pd.DataFrame(SubSector,columns=['subSector'])
    hqLocation = pd.DataFrame(HqLocation,columns=['hqLocation'])
    founded = pd.DataFrame(Founded,columns=['founded'])



    return  pd.concat([ticker, sector, subSector, hqLocation,founded],axis=1)

In [3]:
ticker_data = get_wiki_ticker()
# df["sector"] = row['sector']
# df["subSector"] = row['subSector']
# df["hqLocation"] = row['hqLocation']
# df["founded"] = row['founded']

# Get Price data

In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

ticker = 'NVDA'
url_esg = f"https://query2.finance.yahoo.com/v1/finance/esgChart?symbol={ticker}"
response = requests.get(url_esg, headers=headers)
if response.ok:
    tmp_df_esg = pd.DataFrame(response.json()["esgChart"]["result"][0]["symbolSeries"])

In [5]:
def get_yh_esg(ticker_list):
    
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

    
    #url_esg = "https://query2.finance.yahoo.com/v1/finance/esgChart"
    dataframes_esg = []
    dataframes_price = []

    for ticker in ticker_list:
        # ESG historical data (only changes yearly)
        url_esg = f"https://query2.finance.yahoo.com/v1/finance/esgChart?symbol={ticker}"
        response = requests.get(url_esg, headers=headers)
        if response.ok:
            try:
                tmp_df_esg = pd.DataFrame(response.json()["esgChart"]["result"][0]["symbolSeries"])
            except TypeError:
                continue
            except KeyError:
                continue

            tmp_df_esg["ticker"] = ticker
            tmp_df_esg["timestamp"] = pd.to_datetime(tmp_df_esg["timestamp"], unit="s")
            dataframes_esg.append(tmp_df_esg)

    df_esg = pd.concat(dataframes_esg)


    return df_esg






In [6]:

def get_yh_price(ticker_list, start_period = '1052125800', interval = '1wk'):
    
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    
    dataframes_price = []

    for i, ticker in enumerate(ticker_list):
        print(f'{i} of {ticker_list.shape[0]-1}')

        # Price data
        url_price = f"http://query2.finance.yahoo.com/v8/finance/chart/{ticker}?symbol={ticker}&period1={start_period}&period2=9999999999&interval={interval}"
        response = requests.get(url_price, headers=headers)
        if response.ok:
            try:
                dates = pd.DataFrame({'timestamp':response.json()["chart"]["result"][0]["timestamp"]})

                #low, open ,volume, close, high
                df_quote = pd.DataFrame(response.json()["chart"]["result"][0]["indicators"]["quote"][0])
                # ad_close
                df_adjclose = pd.DataFrame(response.json()["chart"]["result"][0]["indicators"]["adjclose"][0])

                tmp_df_price = pd.concat([dates, df_quote,df_adjclose], axis=1)
                tmp_df_price['timestamp'] = pd.to_datetime(tmp_df_price['timestamp'], unit = "s")

            except TypeError:
                continue
            except KeyError:
                continue

            tmp_df_price["ticker"] = ticker
            dataframes_price.append(tmp_df_price)

    df_price = pd.concat(dataframes_price)

    return  df_price




In [7]:
from pandas_datareader import data
tickers = ['AAPL','AMZN','TSLA','GOOG']
data.get_quote_yahoo(tickers)['marketCap']

AAPL    2572753305600
AMZN    1264042246144
TSLA     902116081664
GOOG    1508192681984
Name: marketCap, dtype: int64

In [8]:
def get_yh_asset_esg(ticker_list):

    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    
    dataframes_asset_profile = []
    dataframes_company_officers = []
    dataframes_esg_info = []

    for ticker in ticker_list:

        # Price data
        url = f"https://query2.finance.yahoo.com/v10/finance/quoteSummary/{ticker}?modules=assetProfile%2CesgScores"
        response = requests.get(url, headers=headers)
        if response.ok:
            try:
                assetProfile = response.json()["quoteSummary"]["result"][0]["assetProfile"]
                #esg_scores_dict = response.json()["quoteSummary"]["result"][0]["esgScores"]
            except TypeError:
                assetProfile = dict()
            except KeyError:
                assetProfile = dict()
            
            try:
                esg_scores_dict = response.json()["quoteSummary"]["result"][0]["esgScores"]
            except TypeError:
                esg_scores_dict = dict()
            except KeyError:
                esg_scores_dict = dict()
                
            
            df_asset_profile = pd.DataFrame({'address':assetProfile.get('address1', None),
                                            'city':assetProfile.get('city', None),
                                            'state':assetProfile.get('state', None),
                                            'zip':assetProfile.get('zip', None),
                                            'country':assetProfile.get('country', None),
                                            'industry':assetProfile.get('industry', None),
                                            'sector':assetProfile.get('sector', None),
                                            'boardRisk':assetProfile.get('boardRisk', None),
                                            'compensationRisk':assetProfile.get('compensationRisk', None),
                                            'shareHolderRightsRisk':assetProfile.get('shareHolderRightsRisk', None),
                                            'overallRisk':assetProfile.get('overallRisk', None),
                                            'ticker':ticker}, 
                                        index = [0])

            dataframes_asset_profile.append(df_asset_profile)

            officers = {'maxAge':list(),
                        'name':list(),
                        'age':list(),
                        'title':list(),
                        'yearBorn':list(),
                        'fiscalYear':list(),
                        'totalPay':list(),
                        'exercisedValue':list(),
                        'unexercisedValue':list()}

            for item in assetProfile['companyOfficers']:
                officers['maxAge'].append(item.get('maxAge', None))
                officers['name'].append(item.get('name', None))
                officers['age'].append(item.get('age', None))
                officers['title'].append(item.get('title', None))
                officers['yearBorn'].append(item.get('yearBorn', None))
                officers['fiscalYear'].append(item.get('fiscalYear', None))

                if not item.get('totalPay', None) is None:
                    officers['totalPay'].append(item['totalPay'].get('raw', None))
                else:
                    officers['totalPay'].append(None)

                if not item.get('exercisedValue', None) is None:
                    officers['exercisedValue'].append(item['exercisedValue'].get('raw', None))
                else:
                    officers['exercisedValue'].append(None)
                    
                if not item.get('unexercisedValue', None) is None:
                    officers['unexercisedValue'].append(item['unexercisedValue'].get('raw', None))
                else:
                    officers['unexercisedValue'].append(None)

        

            officers = pd.DataFrame(officers)#index = list(range(len(assetProfile['companyOfficers'])))
            officers['ticker'] = ticker


            dataframes_company_officers.append(officers)


            dataframes_esg_info.append( pd.DataFrame({'adult':esg_scores_dict.get('adult', None),
                'alcoholic':esg_scores_dict.get('alcoholic', None),
                'animalTesting':esg_scores_dict.get('animalTesting', None),
                'catholic':esg_scores_dict.get('catholic', None),
                'controversialWeapons':esg_scores_dict.get('controversialWeapons', None),
                'smallArms':esg_scores_dict.get('smallArms', None),
                'furLeather':esg_scores_dict.get('furLeather', None),
                'gambling':esg_scores_dict.get('gambling', None),
                'gmo':esg_scores_dict.get('gmo', None),
                'militaryContract':esg_scores_dict.get('militaryContract', None),
                'nuclear':esg_scores_dict.get('nuclear', None),
                'pesticides':esg_scores_dict.get('pesticides', None),
                'palmOil':esg_scores_dict.get('palmOil', None),
                'coal':esg_scores_dict.get('coal', None),
                'tobacco':esg_scores_dict.get('tobacco',None),
                'ticker':ticker}, index = [0])
            )

    df_asset_profiles = pd.concat(dataframes_asset_profile)
    df_company_officers = pd.concat(dataframes_company_officers)
    df_esg_info = pd.concat(dataframes_esg_info)

    return  df_asset_profiles, df_company_officers, df_esg_info
    

In [21]:

ticker_list = ticker_data['ticker'].iloc[250:]
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}


#url_esg = "https://query2.finance.yahoo.com/v1/finance/esgChart"
dataframes_esg = []
dataframes_price = []

for ticker in ticker_list:
    print(ticker)
    # ESG historical data (only changes yearly)
    url_esg = f"https://query2.finance.yahoo.com/v1/finance/esgChart?symbol={ticker}"
    response = requests.get(url_esg, headers=headers)
    if response.ok:
        try:
            tmp_df_esg = pd.DataFrame(response.json()["esgChart"]["result"][0]["symbolSeries"])
        except TypeError:
            continue
        except KeyError:
            continue

        tmp_df_esg["ticker"] = ticker
        tmp_df_esg["timestamp"] = pd.to_datetime(tmp_df_esg["timestamp"], unit="s")
        dataframes_esg.append(tmp_df_esg)
        print(tmp_df_esg.shape)

df_esg = pd.concat(dataframes_esg)



INTC
(90, 6)
ICE
(90, 6)
IBM
(90, 6)
IP
(90, 6)
IPG
(90, 6)
IFF
(90, 6)
INTU
(90, 6)
ISRG
(90, 6)
IVZ
(90, 6)
IPGP
(90, 6)
IQV
(51, 6)
IRM
(90, 6)
JBHT
(90, 6)
JKHY
(67, 6)
J
(90, 6)
JNJ
(90, 6)
JCI
(90, 6)
JPM
(90, 6)
JNPR
(90, 6)
K
(90, 6)
KEY
(90, 6)
KEYS
(77, 6)
KMB
(90, 6)
KIM
(90, 6)
KMI
(90, 6)
KLAC
(90, 6)
KHC
(75, 6)
KR
(90, 6)
LHX
LH
(90, 6)
LRCX
(90, 6)
LW
LVS
(90, 6)
LDOS
(90, 6)
LEN
(90, 6)
LLY
(90, 6)
LNC
(90, 6)
LIN
LYV
LKQ
(90, 6)
LMT
(90, 6)
L
(90, 6)
LOW
(90, 6)
LUMN
(19, 6)
LYB
(90, 6)
MTB
(90, 6)
MRO
(90, 6)
MPC
(90, 6)
MKTX
MAR
(90, 6)
MMC
(90, 6)
MLM
(90, 6)
MAS
(90, 6)
MA
(90, 6)
MTCH
MKC
(90, 6)
MCD
(90, 6)
MCK
(90, 6)
MDT
(90, 6)
MRK
(90, 6)
FB
(90, 6)
MET
(90, 6)
MTD
(90, 6)
MGM
(90, 6)
MCHP
(90, 6)
MU
(90, 6)
MSFT
(90, 6)
MAA
(64, 6)
MRNA
MHK
(90, 6)
MOH
TAP
(90, 6)
MDLZ
(90, 6)
MPWR
MNST
(90, 6)
MCO
(90, 6)
MS
(90, 6)
MOS
(90, 6)
MSI
(90, 6)
MSCI
(90, 6)
NDAQ
(90, 6)
NTAP
(90, 6)
NFLX
(90, 6)
NWL
(90, 6)
NEM
(90, 6)
NWSA
(90, 6)
NWS
NEE
(90, 6)
NLSN
(90, 6)


In [18]:
df_esg1.shape

(19336, 6)

In [22]:
df_esg = pd.concat((df_esg, df_esg1))

In [26]:
len(np.unique(df_esg['ticker']))

453

In [29]:
print("df_esg")
df_esg = get_yh_esg(ticker_data['ticker'])



df_esg


ValueError: No objects to concatenate

In [27]:
print("df_asset_profiles")
df_asset_profiles, df_company_officers, df_esg_info = get_yh_asset_esg(ticker_data['ticker'])

df_asset_profiles


In [28]:
df_asset_profiles.sort_values(by = 'sector')

,address,city,state,zip,country,industry,sector,boardRisk,compensationRisk,shareHolderRightsRisk,overallRisk,ticker
0,101 West Prospect Avenue,Cleveland,OH,44115-1075,United States,Specialty Chemicals,Basic Materials,7,4,8,7,SHW
0,9330 Zionsville Road,Indianapolis,IN,46268-0735,United States,Agricultural Inputs,Basic Materials,5,5,5,4,CTVA
0,2929 Walnut Street,Philadelphia,PA,19104,United States,Agricultural Inputs,Basic Materials,8,6,8,6,FMC
0,The Priestley Centre,Guildford,None,GU2 7XY,United Kingdom,Specialty Chemicals,Basic Materials,6,4,1,4,LIN
0,333 North Central Avenue,Phoenix,AZ,85004-2189,United States,Copper,Basic Materials,5,2,4,3,FCX
...,...,...,...,...,...,...,...,...,...,...,...,...
0,1800 Three Lincoln Centre,Dallas,TX,75240,United States,Utilities—Regulated Gas,Utilities,7,6,7,7,ATO
0,1 Water Street,Camden,NJ,08102-1658,United States,Utilities—Regulated Water,Utilities,1,5,3,1,AWK
0,1 Riverside Plaza,Columbus,OH,43215-2373,United States,Utilities—Regulated Electric,Utilities,3,5,7,8,AEP
0,4 Irving Place,New York,NY,10003,United States,Utilities—Regulated Electric,Utilities,3,7,1,2,ED


In [31]:
print("df_price")
df_price = get_yh_price(ticker_data['ticker'], interval = '1d')

df_price
0 of 503
1 of 503
2 of 503
3 of 503
4 of 503
5 of 503
6 of 503
7 of 503
8 of 503
9 of 503
10 of 503
11 of 503
12 of 503
13 of 503
14 of 503
15 of 503
16 of 503
17 of 503
18 of 503
19 of 503
20 of 503
21 of 503
22 of 503
23 of 503
24 of 503
25 of 503
26 of 503
27 of 503
28 of 503
29 of 503
30 of 503
31 of 503
32 of 503
33 of 503
34 of 503
35 of 503
36 of 503
37 of 503
38 of 503
39 of 503
40 of 503
41 of 503
42 of 503
43 of 503
44 of 503
45 of 503
46 of 503
47 of 503
48 of 503
49 of 503
50 of 503
51 of 503
52 of 503
53 of 503
54 of 503
55 of 503
56 of 503
57 of 503
58 of 503
59 of 503
60 of 503
61 of 503
62 of 503
63 of 503
64 of 503
65 of 503
66 of 503
67 of 503
68 of 503
69 of 503
70 of 503
71 of 503
72 of 503
73 of 503
74 of 503
75 of 503
76 of 503
77 of 503
78 of 503
79 of 503
80 of 503
81 of 503
82 of 503
83 of 503
84 of 503
85 of 503
86 of 503
87 of 503
88 of 503
89 of 503
90 of 503
91 of 503
92 of 503
93 of 503
94 of 503
95 of 503
96 of 503
97 of 503
98 of 503
99 of 503
1

In [32]:
df_price.to_csv('YAHOO_PRICE.csv')

In [30]:
writer = pd.ExcelWriter('YAHOO_PRICE_ESG.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
df_esg.to_excel(writer, sheet_name='ESG')
df_esg_info.to_excel(writer, sheet_name='esg_info')
#df_price.to_excel(writer, sheet_name='PRICE')
df_asset_profiles.to_excel(writer, sheet_name='asset_profiles')
df_company_officers.to_excel(writer, sheet_name='company_officers')

# Close the Pandas Excel writer and output the Excel file.
writer.save()
writer.close()

C:\Users\User\Code\MMDGraph\.mmdvenv\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
